In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
if os.getenv("GOOGLE_API_KEY"):
    print("GOOGLE API KEY loaded successfully")
else: 
    print("GOOGLE API KEY not found")

GOOGLE API KEY loaded successfully


In [10]:
from dotenv import load_dotenv
import os
import subprocess
import mimetypes
from pathlib import Path
import google.generativeai as genai

# Load environment variables
load_dotenv()

# Configure Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def list_files(directory_path: str = "."):
    """List all files and directories in the specified path."""
    try:
        path = Path(directory_path).expanduser()
        if not path.exists():
            return f"Directory '{directory_path}' does not exist"
        
        items = []
        for item in path.iterdir():
            icon = "" if item.is_dir() else ""
            size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
            items.append(f"{icon} {item.name}{size}")
        
        return f"Contents of {path}:\n" + "\n".join(sorted(items))
    except Exception as e:
        return f"Error listing files: {str(e)}"

def read_file(file_path: str):
    """Read text content from a file."""
    try:
        path = Path(file_path).expanduser()
        if not path.exists():
            return f"File '{file_path}' does not exist"
        
        # Check if it's a text file
        mime_type = mimetypes.guess_type(str(path))[0] or ""
        if not mime_type.startswith("text/") and not file_path.endswith(('.txt', '.py', '.md', '.json', '.yaml', '.yml', '.cfg', '.conf')):
            return f"Cannot read non-text file: {file_path}"
        
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        return f"Content of {path}:\n{content[:1000]}{'...' if len(content) > 1000 else ''}"
    except Exception as e:
        return f"Error reading file: {str(e)}"

def play_media(file_path: str):
    """Play audio or video file using system default player."""
    try:
        path = Path(file_path).expanduser()
        if not path.exists():
            return f"File '{file_path}' does not exist"
        
        mime_type = mimetypes.guess_type(str(path))[0] or ""
        
        if mime_type.startswith(("audio/", "video/")):
            # Try VLC first, then mpv, then system default
            players = ["vlc", "mpv", "xdg-open"]
            for player in players:
                try:
                    subprocess.Popen([player, str(path)], 
                                   stdout=subprocess.DEVNULL, 
                                   stderr=subprocess.DEVNULL)
                    return f"Playing '{path.name}' with {player}"
                except FileNotFoundError:
                    continue
            return "No suitable media player found"
        else:
            return f"'{file_path}' is not a media file"
    except Exception as e:
        return f"Error playing media: {str(e)}"

def open_file(file_path: str):
    """Open file with system default application."""
    try:
        path = Path(file_path).expanduser()
        if not path.exists():
            return f"File '{file_path}' does not exist"
        
        subprocess.Popen(["xdg-open", str(path)], 
                        stdout=subprocess.DEVNULL, 
                        stderr=subprocess.DEVNULL)
        return f"Opened '{path.name}' with default application"
    except Exception as e:
        return f"Error opening file: {str(e)}"

def change_directory(directory_path: str):
    """Change current working directory."""
    try:
        path = Path(directory_path).expanduser()
        if not path.exists():
            return f"Directory '{directory_path}' does not exist"
        if not path.is_dir():
            return f"'{directory_path}' is not a directory"
        
        os.chdir(str(path))
        return f"Changed directory to: {os.getcwd()}"
    except Exception as e:
        return f"Error changing directory: {str(e)}"

# Simple approach: Use Python functions directly with Gemini
class FileManagerAgent:
    def __init__(self):
        # Use the model without complex tools - we'll handle function calling manually
        self.model = genai.GenerativeModel(
            model_name="gemini-1.5-flash",
            tools=[list_files, read_file, play_media, open_file, change_directory]
        )
        self.chat = self.model.start_chat()
        self.current_dir = os.getcwd()
    
    def process_request(self, user_input: str):
        """Process user request and execute any function calls."""
        try:
            # Add context about current directory and available functions
            context = f"""Current directory: {os.getcwd()}
                    Available functions:
                    - list_files(directory_path="."): List files and directories
                    - read_file(file_path): Read text content from a file  
                    - play_media(file_path): Play audio/video files
                    - open_file(file_path): Open file with default application
                    - change_directory(directory_path): Change current directory
                    
                    User request: {user_input}"""
            
            response = self.chat.send_message(context)

            
            # Check if there are function calls to execute
            if hasattr(response.candidates[0], 'content') and response.candidates[0].content.parts:
                for part in response.candidates[0].content.parts:
                    if hasattr(part, 'function_call') and part.function_call:
                        function_call = part.function_call
                        print(f"Executing: {function_call.name}")
                        
                        # Execute the function call
                        function_name = function_call.name
                        function_args = dict(function_call.args)
                        
                        # Call the appropriate function
                        if function_name == "list_files":
                            result = list_files(**function_args)
                        elif function_name == "read_file":
                            result = read_file(**function_args)
                        elif function_name == "play_media":
                            result = play_media(**function_args)
                        elif function_name == "open_file":
                            result = open_file(**function_args)
                        elif function_name == "change_directory":
                            result = change_directory(**function_args)
                        else:
                            result = f"Unknown function: {function_name}"
                        
                        # Send function result back to model
                        response = self.chat.send_message(
                            genai.protos.Content(
                                parts=[genai.protos.Part(
                                    function_response=genai.protos.FunctionResponse(
                                        name=function_call.name,
                                        response={"result": result}
                                    )
                                )]
                            )
                        )
                        
                        return {"function_result": result, "ai_response": response.text}
            
            # If no function calls, return the text response
            return {"message": response.text}
            
        except Exception as e:
            return {"error": f"Error processing request: {str(e)}"}

def run_agent():
    """Run the interactive agent loop."""
    print("AI File Manager Agent Started!")
    print("Current directory:", os.getcwd())
    print("Try commands like:")
    print("   - 'list files in current directory'")
    print("   - 'show me what's in Downloads'")
    print("   - 'play music.mp3'")
    print("   - 'open document.pdf'")
    print("   - 'read notes.txt'")
    print("   - 'change to home directory'")
    print("   - Type 'quit' to exit\n")
    
    # Create agent instance
    agent = FileManagerAgent()
    
    while True:
        try:
            user_input = input("You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Goodbye!")
                break
            
            if not user_input:
                continue
            
            print("Processing...")
            result = agent.process_request(user_input)
            
            if "error" in result:
                print(f"{result['error']}")
            elif "function_result" in result:
                print(f"{result['function_result']}")
                # Also show AI response if available
                if "ai_response" in result and result["ai_response"]:
                    print(f"{result['ai_response']}")
            elif "message" in result:
                print(f"{result['message']}")
            else:
                print(f"Result: {result}")
                
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")

# Start the agent
if __name__ == "__main__":
    run_agent()

AI File Manager Agent Started!
Current directory: /home/user1/ai-agents-intro/media-player-agent
Try commands like:
   - 'list files in current directory'
   - 'show me what's in Downloads'
   - 'play music.mp3'
   - 'open document.pdf'
   - 'read notes.txt'
   - 'change to home directory'
   - Type 'quit' to exit



You:  list all files


Processing...
Executing: list_files
Contents of .:
 .env (55 bytes)
 .ipynb_checkpoints
 Untitled.ipynb (618 bytes)
 media-ai-agent.ipynb (13736 bytes)
 memory.json (339 bytes)
 music
 venv
Here's a list of files and directories in the current directory:

- .env
- .ipynb_checkpoints
- Untitled.ipynb
- media-ai-agent.ipynb
- memory.json
- music
- venv




You:  play any one file in music


Processing...
Executing: list_files
Error processing request: Could not convert `part.function_call` to text.


...

Processing...
Executing: play_media
Playing 'fCeeaOfPRkKZDxXca1k-pA_71c148c257bb4265b043ce92af0e87f1_TheAgentLoop_MP4_1080.mp4' with vlc
Playing the file.



You:  play this file 🔥🔥🔥BEST __ RAP __ TRAP HIPHOP VIDEO MIX 2025 DJ BLACKISH X SYD THE DECK FT JAY Z_RICK ROSS_LIL WAYNE(MP3_160K).mp3


Processing...
Executing: play_media
Playing '🔥🔥🔥BEST __ RAP __ TRAP HIPHOP VIDEO MIX 2025 DJ BLACKISH X SYD THE DECK FT JAY Z_RICK ROSS_LIL WAYNE(MP3_160K).mp3' with vlc
Playing the requested file.



You:  quit


Goodbye!
